### Filtering and Mutating Dataset for Training

In [34]:
#!pip3 install langdetect

In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
# Load Dataset
#tweets = pd.read_csv("data/cyberbullying_tweets.csv")
english_tweets = pd.read_csv("data/english_tweets.csv")
non_english_tweets = pd.read_csv("data/non_english_tweets.csv")
#tweets.head(10)

In [4]:
# Looking at one tweet
prototweet = tweets.iloc[2]
print(prototweet["tweet_text"])
print(prototweet["cyberbullying_type"])

@XochitlSuckkks a classy whore? Or more red velvet cupcakes?
not_cyberbullying


In [5]:
# Looking at types of Cyberbullying tweets
tweets["cyberbullying_type"].unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [6]:
tweets.shape #There are 47692 tweets currently

(47692, 2)

In [7]:
# Extracting single tweet to test on
text = prototweet["tweet_text"]
text = text

#### Using Regular Expressions to Mutate Tweet Column

Substituting all @'s with a generic @user token. Reduce data variability

In [8]:
# Subbing usernames with regex
text = re.sub(r'@\w+', '@user', text)
print(text)

@user a classy whore? Or more red velvet cupcakes?


In [9]:
# @user subbing function to apply to whole text column
def sub_usernames(text):
    return re.sub(r'@\w+', '@user', text)

In [10]:
# Mutating text column
tweets["tweet_text"] = tweets["tweet_text"].apply(sub_usernames)
tweets.head(10)

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@user a classy whore? Or more red velvet cupca...,not_cyberbullying
3,"@user meh. :P thanks for the heads up, but no...",not_cyberbullying
4,@user This is an ISIS account pretending to be...,not_cyberbullying
5,"@user @user Yes, the test of god is that good ...",not_cyberbullying
6,Itu sekolah ya bukan tempat bully! Ga jauh kay...,not_cyberbullying
7,Karma. I hope it bites Kat on the butt. She is...,not_cyberbullying
8,@user everything but mostly my priest,not_cyberbullying
9,Rebecca Black Drops Out of School Due to Bully...,not_cyberbullying


In [27]:
#Checking the language of the tweet
from langdetect import detect

def detect_tweet(tweets):
    non_english_tweets = 0
    total_tweets = len(tweets)
    list_non_english = []

    for i in range (total_tweets):
        if (i % 1000 == 0):
            print(f"Stauts: {round((i/total_tweets) * 100, 2)}%")
        try:
            lang = detect(tweets.iloc[i]['tweet_text'])
            if lang != 'en':
                non_english_tweets += 1
                list_non_english.append(i)
        except:
            pass

    print(f"Total tweets: {total_tweets}")
    print(f"Non-English tweets: {non_english_tweets}")
#print(f"Row Numbers of Non-English tweets {list_non_english}")

In [14]:
non_english_tweets = tweets.iloc[list_non_english]
english_tweets = tweets.drop(tweets.index[list_non_english])

print(f"Non-English tweets have: {non_english_tweets.shape[0]} tweets")
print(f"English tweets have: {english_tweets.shape[0]} tweets")

Non-English tweets have: 3115 tweets
English tweets have: 44577 tweets


In [17]:
english_tweets.to_csv('english_tweets.csv', index = False)
non_english_tweets.to_csv('non_english_tweets.csv', index = False)

### Lowercasing and Removing Stop Words

In [7]:
import nltk
import spacy #-- Spacy is generally faster than nltk


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import multiprocessing as mp

In [9]:
nlp = spacy.load('en_core_web_md', disable =['ner', 'parser', 'textcat'])

In [16]:
def remove_stopwords(text):
    doc = nlp(text)
    words = [token.text for token in doc if not token.is_stop]
    return ' '.join(words)

english_tweets['tweets'] = english_tweets["tweet_text"].apply(remove_stopwords)
#english_tweets.head(10)

In [3]:
english_tweets

,tweet_text,cyberbullying_type,tweets
0,"In other words #katandandre, your food was cra...",not_cyberbullying,"words # katandandre , food crapilicious ! # mkr"
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,# aussietv white ? # MKR # theblock # ImACeleb...
2,@user a classy whore? Or more red velvet cupca...,not_cyberbullying,@user classy whore ? red velvet cupcakes ?
3,"@user meh. :P thanks for the heads up, but no...",not_cyberbullying,"@user meh . :P thanks heads , concerned angr..."
4,@user This is an ISIS account pretending to be...,not_cyberbullying,@user ISIS account pretending Kurdish account ...
...,...,...,...
44572,"Black ppl aren't expected to do anything, depe...",ethnicity,"Black ppl expected , depended . free participa..."
44573,Turner did not withhold his disappointment. Tu...,ethnicity,Turner withhold disappointment . Turner called...
44574,I swear to God. This dumb nigger bitch. I have...,ethnicity,swear God . dumb nigger bitch . got bleach hai...
44575,Yea fuck you RT @user: IF YOURE A NIGGER FUCKI...,ethnicity,Yea fuck RT @user : YOURE NIGGER FUCKING UNFOL...


In [4]:
english_tweets['word_count'] = english_tweets['tweets'].apply(lambda x: len(str(x).split()))

In [12]:
def extract_hashtags(text):
    if isinstance(text, str):
        hashtags = re.findall(r'#\s*\w+', text)
        return [tag.strip('#') for tag in hashtags]
    else:
        return []

english_tweets['hashtags'] = english_tweets['tweets'].apply(lambda x: extract_hashtags(x))

In [22]:
english_tweets.to_csv('english_tweets.csv', index = False)